In [2]:
import wandb
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Input, Flatten, LSTM, GRU, Dense, Reshape, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from keras.utils import to_categorical

from types import SimpleNamespace
import keras
import numpy as np
import pandas as pd
import pydicom
import cv2
import re
import sqlite3

import matplotlib.pyplot as plt
import os 
from glob import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
tf.config.run_functions_eagerly(True)
from __datagen__ import DataGenerator
from matplotlib import pyplot
import os
from tqdm import tqdm

ModuleNotFoundError: No module named 'pydicom'

In [2]:
import sys
print(sys.executable)

c:\Users\Daniel\miniconda3\envs\tf\python.exe


In [3]:
#Loaded model
model_unet = load_model(f"C:/Users/Daniel/Desktop/RSNA_Abdominal_Trauma/Submission/U_Net/Unet_Fine_Tune_128_plus_CKP.h5")
#model_unet.summary() U_Net\Unet_Fine_Tune_128_plus_CKP.h5
last_enc_layer = model_unet.get_layer('conv2d_9')
model_encoder = Model(inputs=model_unet.input, outputs=last_enc_layer.output)
model_encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                             

In [4]:
def unet_encoder_for_lstm_input(input_shape, lstm_units):
    unet_encoder = model_encoder
    # Convierte la salida del encoder en una secuencia 1D para la LSTM
    encoder_output = unet_encoder.output
    output_layer = Flatten()(encoder_output)
    model = Model(inputs=unet_encoder.input, outputs=output_layer)
    return model

input_shape = (128, 128, 1)
model2 = unet_encoder_for_lstm_input(input_shape, 100)
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                           

In [5]:
if __name__ == "__main__":
    data = pd.read_csv("C:/Users/Daniel/Desktop/RSNA_Abdominal_Trauma/local_database/train_data_lstm.csv")
    datagen = DataGenerator(data['Patient_id'], data['Series_id'], 64)  
    print(len(data))
    x, y = datagen[0]
    print(x.shape, y.shape)

4711
(64, 64, 128, 128, 1) (64, 13)


In [6]:
for patient in range(len(data)):
    with open(f'D:/Downloads/rsna-2023-abdominal-trauma-detection/volume_data_for_LSTM/{data["Patient_id"][patient]}_{data["Series_id"][patient]}.npy', 'rb') as f:
                    X = np.load(f, allow_pickle=True)
                    y = np.load(f, allow_pickle=True)
    prediction = model2.predict(X)
    X_max = prediction.max()
    X_min = prediction.min()
    normalized_prediction = (prediction - X_min) / (X_max - X_min)
    labels = y
    with open(f'D:/Downloads/rsna-2023-abdominal-trauma-detection/Features_for_LSTM_normalized/{str(data["Patient_id"][patient])}_{str(data["Series_id"][patient])}.npy', 'wb') as f:
                   np.save(f, prediction)
                   np.save(f, labels)
                   print(f'Process finished for patient -> {str(data["Patient_id"][patient])}', f"Final shape saved: {prediction.shape} and labels {labels.shape}")

    

    

C:\Users\Daniel\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 [==============================] - 1s 265ms/step
Process finished for patient -> 25102 Final shape saved: (64, 32768) and labels (13,)
2/2 [==============================] - 1s 264ms/step
Process finished for patient -> 31158 Final shape saved: (64, 32768) and labels (13,)
2/2 [==============================] - 1s 256ms/step
Process finished for patient -> 51177 Final shape saved: (64, 32768) and labels (13,)
2/2 [==============================] - 1s 264ms/step
Process finished for patient -> 56046 Final shape saved: (64, 32768) and labels (13,)
2/2 [==============================] - 1s 261ms/step
Process finished for patient -> 56400 Final shape saved: (64, 32768) and labels (13,)
2/2 [==============================] - 1s 273ms/step
Process finished for patient -> 13646 Final shape saved: (64, 32768) and labels (13,)
2/2 [==============================] - 1s 266ms/step
Process finished for patient -> 54727 Final shape saved: (64, 32768) and labels (13,)
2/2 [======================

In [7]:
np.mean(prediction[0]), max(prediction[0])


(4.7325754, 1035.4313)

In [8]:
for patient in range(len(data[0:10])):
    with open(f'D:/Downloads/rsna-2023-abdominal-trauma-detection/Features_for_LSTM/{data["Patient_id"][patient]}_{data["Series_id"][patient]}.npy', 'rb') as f:
                    X = np.load(f, allow_pickle=True)
                    print(len(X[patient]))
                    y = np.load(f, allow_pickle=True)
                    print(max(X[patient]))

32768
936.7212
32768
958.80396
32768
940.3048
32768
928.19446
32768
1026.4308
32768
766.2595
32768
985.20026
32768
1120.4982
32768
1124.6617
32768
1375.0801
